# Assignment 4
## Description
In this assignment you must read in a file of metropolitan regions and associated sports teams from [assets/wikipedia_data.html](assets/wikipedia_data.html) and answer some questions about each metropolitan region. Each of these regions may have one or more teams from the "Big 4": NFL (football, in [assets/nfl.csv](assets/nfl.csv)), MLB (baseball, in [assets/mlb.csv](assets/mlb.csv)), NBA (basketball, in [assets/nba.csv](assets/nba.csv) or NHL (hockey, in [assets/nhl.csv](assets/nhl.csv)). Please keep in mind that all questions are from the perspective of the metropolitan region, and that this file is the "source of authority" for the location of a given sports team. Thus teams which are commonly known by a different area (e.g. "Oakland Raiders") need to be mapped into the metropolitan region given (e.g. San Francisco Bay Area). This will require some human data understanding outside of the data you've been given (e.g. you will have to hand-code some names, and might need to google to find out where teams are)!

For each sport I would like you to answer the question: **what is the win/loss ratio's correlation with the population of the city it is in?** Win/Loss ratio refers to the number of wins over the number of wins plus the number of losses. Remember that to calculate the correlation with [`pearsonr`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.pearsonr.html), so you are going to send in two ordered lists of values, the populations from the wikipedia_data.html file and the win/loss ratio for a given sport in the same order. Average the win/loss ratios for those cities which have multiple teams of a single sport. Each sport is worth an equal amount in this assignment (20%\*4=80%) of the grade for this assignment. You should only use data **from year 2018** for your analysis -- this is important!

## Notes

1. Do not include data about the MLS or CFL in any of the work you are doing, we're only interested in the Big 4 in this assignment.
2. I highly suggest that you first tackle the four correlation questions in order, as they are all similar and worth the majority of grades for this assignment. This is by design!
3. It's fair game to talk with peers about high level strategy as well as the relationship between metropolitan areas and sports teams. However, do not post code solving aspects of the assignment (including such as dictionaries mapping areas to teams, or regexes which will clean up names).
4. There may be more teams than the assert statements test, remember to collapse multiple teams in one city into a single value!

## Question 1
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NHL** using **2018** data.

In [1]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

nhl_df=pd.read_csv("assets/nhl.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

def nhl_final():
    nhl_df=pd.read_csv("assets/nhl.csv")
    cities=pd.read_html("assets/wikipedia_data.html")[1]
    cities=cities.iloc[:-1,[0,3,5,6,7,8]]

    population_by_region = [] # pass in metropolitan area population from cities
    win_loss_by_region = [] # pass in win/loss ratio from nhl_df in the same order as cities["Metropolitan area"]

    nhl_df = nhl_df.where(nhl_df['year']==2018).dropna(how='all')
    nhl_df['team'] = nhl_df['team'].str.replace('[^\w ]', '')
    nhl_df = nhl_df[~nhl_df['team'].str.contains('(Division)')]
    nhl_df['W'] = pd.to_numeric(nhl_df['W'])
    nhl_df['L'] = pd.to_numeric(nhl_df['L'])
    nhl_df['wl_ratio'] = nhl_df['W'] / (nhl_df['W'] + nhl_df['L'])

    cities['NHL'] = cities['NHL'].str.extract(r'((?:^|(?<= ))[A-Z][A-Za-z]+)', expand=False) 
        #Above eliminates bracketed notes at the end of each team description, e.g. [note 3]
    cities.rename(columns={'Metropolitan area':'Metro'}, inplace=True)
    metrolist = cities.Metro.unique().tolist()
    nhl_df['metro'] = nhl_df['team'].apply(lambda x: ''.join([city for city in metrolist if city in x]))
        #creates new metro column to nhl_df and populates with known areas from cities df, need to hand code missing values
    nhl_df['team'] = nhl_df['team'].str.strip()
    nhl_df = nhl_df.sort_values(by=['team'])
    nhl_df.reset_index(drop=True,inplace=True)
        #need to sort and reset index in order to hand code missing metro areas for teams
    nhl_df = nhl_df[['team','W', 'L', 'year', 'wl_ratio', 'metro']] #select relevant columns
    nhl_df['metro'].replace(r'^\s*$', np.nan, regex=True, inplace=True) #apply NaN to missing metro data
    citydict = {0:'Los Angeles',1:'Phoenix',5:'Raleigh',7:'Denver',9:'Dallas–Fort Worth',12:'Miami–Fort Lauderdale',14:'Minneapolis–Saint Paul',17:'New York City',18: 'New York City', 19: 'New York City', 23:'San Francisco Bay Area', 24:'St. Louis',25:'Tampa Bay Area',28:'Las Vegas',29:'Washington, D.C.'}
    nhl_df['metro'].fillna(citydict, inplace=True)
        #fill missing metro data with dictionary values, dict keys being reference to index that was reset earlier
    nhl_df['avgWL'] = nhl_df.groupby('metro')['wl_ratio'].transform('mean')
        #calculates average wl_ratio by metro area, results are entered into AvgWL column more than once if there is more than 1 team per metro area
    cities['Population (2016 est.)[8]'] = pd.to_numeric(cities['Population (2016 est.)[8]'])
        #needed else Dallas-Fort Worth population becomes NaN in next step, i.e. series was not all integer type
    nhl_df['Population'] = nhl_df.metro.map(cities.set_index('Metro')['Population (2016 est.)[8]'].to_dict())
    #print(nhl_df)

    nhl_final = nhl_df[['metro','avgWL','Population']]
    nhl_final = nhl_final.drop_duplicates(subset=['metro'])
    nhl_final = nhl_final.sort_values(by=['metro'])
    nhl_final = nhl_final.set_index('metro',drop=True)
    #print(nhl_final)
    
    return nhl_final

nhl = nhl_final()

def nhl_correlation(): 
    # YOUR CODE HERE
    #raise NotImplementedError()

    population_by_region = nhl['Population'] # pass in metropolitan area population from cities
    win_loss_by_region = nhl['avgWL'] # pass in win/loss ratio from nhl_df in the same order as cities["Metropolitan area"]

    nhl_corr = stats.pearsonr(population_by_region, win_loss_by_region)
    
    assert len(population_by_region) == len(win_loss_by_region), "Q1: Your lists must be the same length"
    assert len(population_by_region) == 28, "Q1: There should be 28 teams being analysed for NHL"
    
    return nhl_corr[0]

nhl_correlation()

0.012486162921209907

## Question 2
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NBA** using **2018** data.

In [2]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

nba_df=pd.read_csv("assets/nba.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

def nba_final():
    population_by_region = [] # pass in metropolitan area population from cities
    win_loss_by_region = [] # pass in win/loss ratio from nba_df in the same order as cities["Metropolitan area"]

    nba_df=pd.read_csv("assets/nba.csv")
    cities=pd.read_html("assets/wikipedia_data.html")[1]
    cities=cities.iloc[:-1,[0,3,5,6,7,8]]

    nba_df['year'] = pd.to_numeric(nba_df['year'])
    nba_df = nba_df.where(nba_df['year']==2018).dropna(how='all')
    #nba_df['team'] = nba_df['team'].str.replace('[^\w ]', '')
    nba_df['team'] = nba_df['team'].str.extract(r'((?:^|(?<= ))[A-Z][ A-Za-z]+)', expand=False)
    nba_df['team'] = nba_df['team'].str.strip()
    nba_df['W'] = pd.to_numeric(nba_df['W'])
    nba_df['L'] = pd.to_numeric(nba_df['L'])
    nba_df['wl_ratio'] = nba_df['W'] / (nba_df['W'] + nba_df['L'])
    cities['NBA'] = cities['NBA'].str.extract(r'((?:^|(?<= ))[A-Z][A-Za-z]+)', expand=False) 
        #Above eliminates bracketed notes at the end of each team description, e.g. [note 3]
    cities.rename(columns={'Metropolitan area':'Metro'}, inplace=True)
    metrolist = cities.Metro.unique().tolist()
    nba_df['metro'] = nba_df['team'].apply(lambda x: ''.join([city for city in metrolist if city in x]))
        #creates new metro column to nhl_df and populates with known areas from cities df, need to hand code missing values
    nba_df['team'] = nba_df['team'].str.strip()
    nba_df = nba_df.sort_values(by=['team'])
    nba_df.reset_index(drop=True,inplace=True)
        #need to sort and reset index in order to hand code missing metro areas for teams
    nba_df = nba_df[['team','W', 'L', 'year', 'wl_ratio', 'metro']] #select relevant columns
    nba_df['metro'].replace(r'^\s*$', np.nan, regex=True, inplace=True) #apply NaN to missing metro data
    citydict = {2:'New York City',6:'Dallas–Fort Worth',9:'San Francisco Bay Area',11:'Indianapolis',15:'Miami–Fort Lauderdale',17:'Minneapolis–Saint Paul',19:'New York City',28:'Salt Lake City',29: 'Washington, D.C.'}
        #note that Metro names uses a longer dash '–' vs '-' in the cities DF so the dict above uses...
        #the longer dash, this took a while to figure out, it was returning NaN for Dallas population...
        #because it was not properly mapping below to create nba_df['Population'] column due to incorrect sized dash used in the hand written dict
    nba_df['metro'].fillna(citydict, inplace=True)
        #fill missing metro data with dictionary values, dict keys being reference to index that was reset earlier
    nba_df['avgWL'] = nba_df.groupby('metro')['wl_ratio'].transform('mean')
        #calculates average wl_ratio by metro area, results are entered into AvgWL column more than once if there is more than 1 team per metro area
    cities['Population (2016 est.)[8]'] = pd.to_numeric(cities['Population (2016 est.)[8]'])
        #needed else Dallas-Fort Worth population becomes NaN in next step, i.e. series was not all integer type
    nba_df['Population'] = nba_df.metro.map(cities.set_index('Metro')['Population (2016 est.)[8]'].to_dict())
    #print(nba_df)

    nba_final = nba_df[['metro','avgWL','Population']]
    nba_final = nba_final.drop_duplicates(subset=['metro'])
    nba_final = nba_final.sort_values(by=['metro'])
    nba_final = nba_final.set_index('metro',drop=True)
    #print(nba_final)

    return nba_final

def nba_correlation():
    # YOUR CODE HERE
    
    nba = nba_final()

    population_by_region = nba['Population'] # pass in metropolitan area population from cities
    win_loss_by_region = nba['avgWL'] # pass in win/loss ratio from nhl_df in the same order as cities["Metropolitan area"]

    nba_corr = stats.pearsonr(population_by_region, win_loss_by_region)
    nba_corr[0]
    
    assert len(population_by_region) == len(win_loss_by_region), "Q2: Your lists must be the same length"
    assert len(population_by_region) == 28, "Q2: There should be 28 teams being analysed for NBA"

    #raise NotImplementedError()
    return nba_corr[0]

nba_correlation()

-0.17657160252844617

## Question 3
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **MLB** using **2018** data.

In [3]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

mlb_df=pd.read_csv("assets/mlb.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

def mlb_final():
    mlb_df=pd.read_csv("assets/mlb.csv")
    cities=pd.read_html("assets/wikipedia_data.html")[1]
    cities=cities.iloc[:-1,[0,3,5,6,7,8]]

    population_by_region = [] # pass in metropolitan area population from cities
    win_loss_by_region = [] # pass in win/loss ratio from mlb_df in the same order as cities["Metropolitan area"]

    population_by_region = [] # pass in metropolitan area population from cities
    win_loss_by_region = [] # pass in win/loss ratio from nba_df in the same order as cities["Metropolitan area"]

    mlb_df['year'] = pd.to_numeric(mlb_df['year'])
    mlb_df = mlb_df.where(mlb_df['year']==2018).dropna(how='all')
    mlb_df['team'] = mlb_df['team'].str.strip()
    mlb_df['W'] = pd.to_numeric(mlb_df['W'])
    mlb_df['L'] = pd.to_numeric(mlb_df['L'])
    mlb_df['wl_ratio'] = mlb_df['W'] / (mlb_df['W'] + mlb_df['L'])
    cities['MLB'] = cities['MLB'].str.extract(r'((?:^|(?<= ))[A-Z][A-Za-z]+)', expand=False) 
        #Above eliminates bracketed notes at the end of each team description, e.g. [note 3]
    cities.rename(columns={'Metropolitan area':'Metro'}, inplace=True)

    metrolist = cities.Metro.unique().tolist()
    mlb_df['metro'] = mlb_df['team'].apply(lambda x: ''.join([city for city in metrolist if city in x]))
        #creates new metro column to nhl_df and populates with known areas from cities df, need to hand code missing values
    mlb_df['team'] = mlb_df['team'].str.strip()
    mlb_df = mlb_df.sort_values(by=['team'])
    mlb_df.reset_index(drop=True,inplace=True)
        #need to sort and reset index in order to hand code missing metro areas for teams
    mlb_df = mlb_df[['team','W', 'L', 'year', 'wl_ratio', 'metro']] #select relevant columns
    mlb_df['metro'].replace(r'^\s*$', np.nan, regex=True, inplace=True) #apply NaN to missing metro data
    citydict = {0:'Phoenix',8:'Denver',14:'Miami–Fort Lauderdale',16:'Minneapolis–Saint Paul',17:'New York City',18:'New York City',19:'San Francisco Bay Area',23:'San Francisco Bay Area',26: 'Tampa Bay Area',27:'Dallas–Fort Worth',29:'Washington, D.C.'}
        #note that Metro names uses a longer dash '–' vs '-' in the cities DF so the dict above uses...
        #the longer dash. This took time to figure out, it was returning NaN for Dallas population...
        #because it was not properly mapping below to create nba_df['Population'] column due to incorrect sized dash used in the hand typed dict
    mlb_df['metro'].fillna(citydict, inplace=True)
        #fill missing metro data with dictionary values, dict keys being reference to index that was reset earlier
    mlb_df['avgWL'] = mlb_df.groupby('metro')['wl_ratio'].transform('mean')
        #calculates average wl_ratio by metro area, results are entered into AvgWL column more than once if there is more than 1 team per metro area
    cities['Population (2016 est.)[8]'] = pd.to_numeric(cities['Population (2016 est.)[8]'])
        #needed else Dallas-Fort Worth population becomes NaN in next step, i.e. series was not all integer type
    mlb_df['Population'] = mlb_df.metro.map(cities.set_index('Metro')['Population (2016 est.)[8]'].to_dict())
    #print(mlb_df)

    mlb_final = mlb_df[['metro','avgWL','Population']]
    mlb_final = mlb_final.drop_duplicates(subset=['metro'])
    mlb_final = mlb_final.sort_values(by=['metro'])
    mlb_final = mlb_final.set_index('metro',drop=True)
    #print(mlb_final)

    return mlb_final

def mlb_correlation(): 
    # YOUR CODE HERE

    mlb = mlb_final()

    population_by_region = mlb['Population'] # pass in metropolitan area population from cities
    win_loss_by_region = mlb['avgWL'] # pass in win/loss ratio from nhl_df in the same order as cities["Metropolitan area"]

    mlb_corr = stats.pearsonr(population_by_region, win_loss_by_region)
    mlb_corr[0]
    
    assert len(population_by_region) == len(win_loss_by_region), "Q3: Your lists must be the same length"
    assert len(population_by_region) == 26, "Q3: There should be 26 teams being analysed for MLB"

    #raise NotImplementedError()
    return mlb_corr[0]

mlb_correlation()

0.15027698302669307

## Question 4
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NFL** using **2018** data.

In [4]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

nfl_df=pd.read_csv("assets/nfl.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

def nfl_final():
    population_by_region = [] # pass in metropolitan area population from cities
    win_loss_by_region = [] # pass in win/loss ratio from nfl_df in the same order as cities["Metropolitan area"]

    nfl_df=pd.read_csv("assets/nfl.csv")
    cities=pd.read_html("assets/wikipedia_data.html")[1]
    cities=cities.iloc[:-1,[0,3,5,6,7,8]]

    nfl_df['year'] = pd.to_numeric(nfl_df['year'])
    nfl_df = nfl_df.where(nfl_df['year']==2018).dropna(how='all')
    nfl_df = nfl_df[~nfl_df['W'].str.contains('(FC)')]
    nfl_df['team'] = nfl_df['team'].str.extract(r'((?:^|(?<= ))[A-Z][ A-Za-z][ \w]+)', expand=False)
    nfl_df['team'] = nfl_df['team'].str.strip()
    nfl_df['W'] = pd.to_numeric(nfl_df['W'])
    nfl_df['L'] = pd.to_numeric(nfl_df['L'])
    nfl_df['wl_ratio'] = nfl_df['W'] / (nfl_df['W'] + nfl_df['L'])
    cities['NFL'] = cities['NFL'].str.extract(r'((?:^|(?<= ))[A-Z][A-Za-z]+)', expand=False) 
        #Above eliminates bracketed notes at the end of each team description, e.g. [note 3]
    cities.rename(columns={'Metropolitan area':'Metro'}, inplace=True)

    metrolist = cities.Metro.unique().tolist()
    nfl_df['metro'] = nfl_df['team'].apply(lambda x: ''.join([city for city in metrolist if city in x]))
        #creates new metro column to nhl_df and populates with known areas from cities df, need to hand code missing values
    nfl_df['team'] = nfl_df['team'].str.strip()
    nfl_df = nfl_df.sort_values(by=['team'])
    nfl_df.reset_index(drop=True,inplace=True)
        #need to sort and reset index in order to hand code missing metro areas for teams
    nfl_df = nfl_df[['team','W', 'L', 'year', 'wl_ratio', 'metro']] #select relevant columns
    nfl_df['metro'].replace(r'^\s*$', np.nan, regex=True, inplace=True) #apply NaN to missing metro data
    citydict = {0:'Phoenix',4:'Charlotte',8:'Dallas–Fort Worth',18:'Miami–Fort Lauderdale',19:'Minneapolis–Saint Paul',20:'Boston',22:'New York City',23:'New York City',24:'San Francisco Bay Area',27:'San Francisco Bay Area',29:'Tampa Bay Area',30:'Nashville',31:'Washington, D.C.'}
        #note that Metro names uses a longer dash '–' vs shorter '-' in the cities DF so the dict above uses...
        #the longer dash. This took time to figure out, it was returning NaN for Dallas population...
        #because it was not properly mapping below to create nba_df['Population'] column due to incorrect sized dash used in the hand typed dict
    nfl_df['metro'].fillna(citydict, inplace=True)
        #fill missing metro data with dictionary values, dict keys being reference to index that was reset earlier
    nfl_df['avgWL'] = nfl_df.groupby('metro')['wl_ratio'].transform('mean')
        #calculates average wl_ratio by metro area, results are entered into AvgWL column more than once if there is more than 1 team per metro area
    cities['Population (2016 est.)[8]'] = pd.to_numeric(cities['Population (2016 est.)[8]'])
        #needed else Dallas-Fort Worth population becomes NaN in next step, i.e. series was not all integer type
    nfl_df['Population'] = nfl_df.metro.map(cities.set_index('Metro')['Population (2016 est.)[8]'].to_dict())
    #print(nfl_df)

    nfl_final = nfl_df[['metro','avgWL','Population']]
    nfl_final = nfl_final.drop_duplicates(subset=['metro'])
    nfl_final = nfl_final.sort_values(by=['metro'])
    nfl_final = nfl_final.set_index('metro',drop=True)
    #print(nfl_final)

    return nfl_final

def nfl_correlation(): 
    # YOUR CODE HERE
    
    nfl = nfl_final()

    population_by_region = nfl['Population'] # pass in metropolitan area population from cities
    win_loss_by_region = nfl['avgWL'] # pass in win/loss ratio from nhl_df in the same order as cities["Metropolitan area"]

    nfl_corr = stats.pearsonr(population_by_region, win_loss_by_region)
    nfl_corr[0]
    
    assert len(population_by_region) == len(win_loss_by_region), "Q4: Your lists must be the same length"
    assert len(population_by_region) == 29, "Q4: There should be 29 teams being analysed for NFL"

    #raise NotImplementedError()
    return nfl_corr[0]

nfl_correlation()

0.004922112149349393

## Question 5
In this question I would like you to explore the hypothesis that **given that an area has two sports teams in different sports, those teams will perform the same within their respective sports**. How I would like to see this explored is with a series of paired t-tests (so use [`ttest_rel`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.ttest_rel.html)) between all pairs of sports. Are there any sports where we can reject the null hypothesis? Again, average values where a sport has multiple teams in one region. Remember, you will only be including, for each sport, cities which have teams engaged in that sport, drop others as appropriate. This question is worth 20% of the grade for this assignment.

In [5]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

mlb_df=pd.read_csv("assets/mlb.csv")
nhl_df=pd.read_csv("assets/nhl.csv")
nba_df=pd.read_csv("assets/nba.csv")
nfl_df=pd.read_csv("assets/nfl.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

def sports_team_performance():
    # YOUR CODE HERE
    
    # Note: p_values is a full dataframe, so df.loc["NFL","NBA"] should be the same as df.loc["NBA","NFL"] and
    # df.loc["NFL","NFL"] should return np.nan
    sports = ['NFL', 'NBA', 'NHL', 'MLB']
    p_values = pd.DataFrame({k:np.nan for k in sports}, index=sports)
    
    nfl = nfl_final()
    nba = nba_final()
    nhl = nhl_final()
    mlb = mlb_final()
    sport_df = [nfl, nba, nhl, mlb]

    p_vals = []
    for sport_i in sport_df:
        for sport_j in sport_df:
            merge_df = pd.merge(sport_i, sport_j, on = ['metro'])     #merged df has 4 columns with index being metro area: avgWL sport 1, population, avgWL sport 2, population
            ttest = stats.ttest_rel(merge_df.iloc[:, 0], merge_df.iloc[:, 2])    #feeding into ttest_rel the avgWL columns for each sport from line above
            p_vals.append(ttest[1])     #need the second value of the returned tuple which is the 2-sided p-value then append to list

    p_array = np.array(p_vals)
    p_array = p_array.reshape(4,4)
    p_values = pd.DataFrame(data=p_array, index=sports, columns=sports)
    
    assert abs(p_values.loc["NBA", "NHL"] - 0.02) <= 1e-2, "The NBA-NHL p-value should be around 0.02"
    assert abs(p_values.loc["MLB", "NFL"] - 0.80) <= 1e-2, "The MLB-NFL p-value should be around 0.80"
    #raise NotImplementedError()
    return p_values

sports_team_performance()

,NFL,NBA,NHL,MLB
NFL,NaN,0.941792,0.030883,0.802069
NBA,0.941792,NaN,0.022297,0.950540
NHL,0.030883,0.022297,NaN,0.000708
MLB,0.802069,0.950540,0.000708,NaN
